In [ ]:
# === IMPORTS === #
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import nltk
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import yfinance as yf
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VaderAnalyzer
import matplotlib.pyplot as plt

# === LOAD ENVIRONMENT VARIABLES === #
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# === SET UP NEWS API CALL === #
url = 'https://newsapi.org/v2/everything'
parameters = {
    'q': 'NVDA',  # Use stock ticker of interest
    'apiKey': api_key,
    'language': 'en',
    'pageSize': 15
}

# === MAKE NEWS API REQUEST === #
response = requests.get(url, params=parameters)
if response.status_code != 200:
    print("Failed to retrieve news:", response.status_code)
    exit()

# === PARSE NEWS ARTICLES === #
data = response.json()
news_df = pd.DataFrame(data['articles'])
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt']).dt.date

# === SETUP NLP TOOLS === #
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
sia = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

# === TEXT CLEANING FUNCTION === #
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return " ".join([word for word in word_tokenize(text) if word not in stop_words])

# === CLEAN TITLES AND APPLY VADER SENTIMENT === #
news_df['cleaned_title'] = news_df['title'].apply(clean_text)
news_df['sentiment'] = news_df['cleaned_title'].apply(lambda x: sia.polarity_scores(x)['compound'])

# === LOAD FINANCIAL SENTIMENT DICTIONARY === #
lm_df = pd.read_csv(r'C:\Users\roman\Downloads\Loughran-McDonald_MasterDictionary_1993-2023.csv')
lm_positive = set(lm_df[lm_df['Positive'] > 0]['Word'].str.lower())
lm_negative = set(lm_df[lm_df['Negative'] > 0]['Word'].str.lower())

# === ADJUST SENTIMENT BASED ON FINANCIAL CONTEXT === #
def financial_sentiment_adjustment(text, sentiment_score):
    words = text.lower().split()
    pos_score, neg_score = 0, 0
    for word in words:
        if word in lm_positive:
            pos_score += 1
        elif word in lm_negative:
            neg_score += 1
    adjusted = sentiment_score + (pos_score - neg_score) * 0.1
    return max(-1, min(1, adjusted))

news_df['adjusted_sentiment'] = news_df.apply(
    lambda row: financial_sentiment_adjustment(row['cleaned_title'], row['sentiment']), axis=1)

# === AGGREGATE SENTIMENT BY DATE === #
daily_adjusted_sentiment = news_df.groupby('publishedAt')['adjusted_sentiment'].mean().reset_index()

# === LOAD STOCK DATA === #
stock_data = yf.download('NVDA', start='2025-01-01', end='2025-02-01')
stock_data = stock_data[['Close']].reset_index()
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# === MERGE NEWS WITH STOCK DATA === #
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])
merged_data = pd.merge(news_df, stock_data, left_on='publishedAt', right_on='Date', how='inner')

# === FEATURE ENGINEERING === #
merged_data.set_index(pd.to_datetime(merged_data['publishedAt']), inplace=True)
merged_data['lagged_sentiment'] = merged_data['adjusted_sentiment'].shift(1)
merged_data['7_day_MA'] = merged_data['Close'].rolling(window=7).mean()
merged_data.fillna(method='ffill', inplace=True)

# === NORMALIZE FEATURES === #
scaler = StandardScaler()
merged_data[['adjusted_sentiment', 'Close', '7_day_MA']] = scaler.fit_transform(
    merged_data[['adjusted_sentiment', 'Close', '7_day_MA']]
)

# === MODEL TRAINING - RIDGE REGRESSION === #
X = merged_data[['adjusted_sentiment', 'Close', '7_day_MA']]
y = merged_data['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
y_train_imputed = y_train.values.reshape(-1, 1)
y_test_imputed = y_test.values.reshape(-1, 1)

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_imputed, y_train_imputed)

ridge_r_squared = ridge_model.score(X_test_imputed, y_test_imputed)
print("Ridge R-squared:", ridge_r_squared)

cv_scores = cross_val_score(ridge_model, X_train_imputed, y_train_imputed.ravel(), cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Average Cross-Validation Score:", np.mean(cv_scores))

# === FEATURE SELECTION === #
feature_selector = RFE(ridge_model, n_features_to_select=1)
feature_selector.fit(X_train_imputed, y_train_imputed.ravel())
selected_features = X.columns[feature_selector.support_]
print("Selected Features by RFE:", selected_features)

# === FINAL MODEL === #
final_model = Ridge(alpha=1.0)
final_model.fit(X_train_imputed[:, feature_selector.support_], y_train_imputed)
final_r_squared = final_model.score(X_test_imputed[:, feature_selector.support_], y_test_imputed)
print("Final R-squared with selected features:", final_r_squared)

# === HOURLY MOVEMENT PREDICTION (LOGISTIC REGRESSION) === #
merged_data_hourly = merged_data.resample('H').mean().fillna(method='ffill')
merged_data_hourly['Movement'] = (merged_data_hourly['Close'].shift(-1) > merged_data_hourly['Close']).astype(int)
merged_data_hourly['lagged_sentiment'] = merged_data_hourly['adjusted_sentiment'].shift(1)
merged_data_hourly.dropna(inplace=True)

X_hourly = merged_data_hourly[['adjusted_sentiment', '7_day_MA', 'lagged_sentiment']]
y_hourly = merged_data_hourly['Movement']
split_index = int(len(X_hourly) * 0.8)
X_train_hourly, X_test_hourly = X_hourly.iloc[:split_index], X_hourly.iloc[split_index:]
y_train_hourly, y_test_hourly = y_hourly.iloc[:split_index], y_hourly.iloc[split_index:]

X_train_hourly_imputed = imputer.fit_transform(X_train_hourly)
X_test_hourly_imputed = imputer.transform(X_test_hourly)

model_lr_hourly = LogisticRegression()
model_lr_hourly.fit(X_train_hourly_imputed, y_train_hourly)
y_pred_lr_hourly = model_lr_hourly.predict(X_test_hourly_imputed)

print("Hourly Logistic Regression Accuracy:", accuracy_score(y_test_hourly, y_pred_lr_hourly))
print("Hourly Classification Report:\n", classification_report(y_test_hourly, y_pred_lr_hourly))

# === VISUALIZATION: PREDICTED VS ACTUAL PRICES === #
ridge_predictions = ridge_model.predict(X_test_imputed)
plt.figure(figsize=(12, 6))
plt.plot(y_test_imputed, label='Actual Stock Price', color='blue', alpha=0.7)
plt.plot(ridge_predictions, label='Predicted Stock Price', color='red', alpha=0.7)
plt.title('Ridge Regression: Actual vs Predicted Stock Prices')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()